In [61]:
# backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/notebook/playground.ipynb
import sys
import os

# Locate the "src" dynamically from notebook location
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

SRC_DIR = os.path.join(PROJECT_ROOT, "src")

# Debug prints to check
print(f"Current Notebook Dir: {NOTEBOOK_DIR}")
print(f"Project Root (agent_tailored_cover_letter): {PROJECT_ROOT}")
print(f"Adding SRC_DIR to sys.path: {SRC_DIR}")

sys.path.append(SRC_DIR)




Current Notebook Dir: /home/mangabat/projects/portofolio/backend/aiml_models/agent_teams/agent_tailored_cover_letter/notebook
Project Root (agent_tailored_cover_letter): /home/mangabat/projects/portofolio/backend/aiml_models/agent_teams/agent_tailored_cover_letter
Adding SRC_DIR to sys.path: /home/mangabat/projects/portofolio/backend/aiml_models/agent_teams/agent_tailored_cover_letter/src


In [62]:
import httpx
from typing import List, Literal, Dict

class CorrectionsClient:
    def __init__(self) -> None:
        self.base_url = "http://localhost:8010/corrections"

    def fetch_corrections(self, correction_type: Literal["word", "sentence", "skill"]) -> List[Dict[str, str]]:
        response = httpx.get(self.base_url, params={"correction_type": correction_type})
        response.raise_for_status()
        return response.json()  # Ensure it returns a list of dictionaries
    
skills = CorrectionsClient().fetch_corrections("skill")


In [63]:
job_description = """
Data Scientist til Fraud Detection & AI Solutions
Kan du dykke dybt i data? Har du styr på Machine Learning-modeller, SQL og Python? Og vil du arbejde med landets mest interessante datagrundlag? Så er du den Data Scientist, vi søger til afdelingen Fraud Detection & AI Solutions.
Vil du være med til at fremtidssikre velfærdssamfundet med teknologier som ML, NLP og Computer Vision?
I Digital Solutions får vi velfærden til at fungere. Vi spiller en afgørende rolle i digitaliseringen af det danske samfund, fordi vi står bag de systemer, der får to ud af tre velfærdskroner ud til danskerne. 
Her er vi lige nu på udkig efter en Data Scientist til afdelingen Fraud Detection & AI Solutions, hvor vi bl.a. sikrer en koordineret indsats i forhold til kontrol af fejludbetalinger og snyd med offentlige ydelser. 
Derudover spiller vi en vigtig rolle i arbejdet med effektivisering af forretningens processer ved brug af teknologier som Machine Learning, NLP, Computer vision og lignende teknologier. Siden sidste år har vi kortlagt et (kæmpe)stort NLP-potentiale på tværs af ATP, og nu arbejder vi på at udvikle og implementere løsningerne, så vi sikrer en effektiv udbetaling.
I jobbet som Data Scientist er det mere konkret dig, der: 
Forstår forretningens behov på tæt hold.
For at udvikle de bedste løsninger er det vigtigt at forstå forretningens behov. Derfor kommer du til at arbejde tæt sammen med forskellige teams for at afdække deres behov og omsætte dem til datadrevne løsninger.
Udvikler avancerede modeller i Python
Du er med til at udvikle statistiske modeller i Python, der er baseret på Machine Learning ved brug af både træningsdata og unsupervised metoder.
Vedligeholder og monitorerer vores modeller
For at sikre, at vores modeller fungerer optimalt, bliver du ansvarlig for løbende monitorering og vedligeholdelse. Her holder bl.a. øje med, hvorvidt modellerne opfører sig som forventet og tilpasser dem efter behov.
Er med fra udvikling til produktion
Vi arbejder med best practices fra softwareudvikling for at gøre overgangen fra udvikling til produktion så gnidningsfri som muligt. Du er derfor med til at sikre, at vores løsninger bringes sikkert fra udvikling til produktion i vores scrum-setup.
Udvikler og implementerer NLP-løsninger
Sidst, men ikke mindst, kommer du til at spille en central rolle i udviklingen af NLP-løsninger og sørger for, at de bliver produktionssat og taget i brug af vores interne kunder.
Har du styr på SQL og Python?
Der kan være flere veje til rollen som Data Scientist, men vi forestiller os, at du har en relevant kandidatgrad inden for matematik, statistik, fysik, computer science, ingeniørvidenskab eller lignende. Hvis du har et par års erfaring, er det en fordel.
Derudover er du:
erfaren når det kommer til dataanalyse og udvikling af Machine Learning-modeller
en haj til Python, SQL og lignende (en fordel, ikke et krav)
med til at skabe resultater i samarbejde med andre
god til at finde enkle løsninger på komplekse udfordringer.
Vil du være en del af et unikt it-fagligt fællesskab?
I Fraud Detection & AI er vi en del af enheden Data i Digital Solutions. Vi brænder for data og de muligheder, data kan skabe. Som landets største udbetalingshus administrerer ATP to ud af tre velfærdskroner i Danmark. 
Vi arbejder med et unikt datagrundlag, som giver os særlige muligheder – både for at lave dybdegående analyser og for at udvikle datadrevne løsninger.
Du bliver en del af en afdeling med mere end 40 dygtige kolleger, der arbejder som Data Scientists, Software Developers, Data Analysts og forretningsansvarlige. Du kommer især til at arbejde tæt sammen med afdelingens dygtige tekniske specialister, som arbejder med Data Science og softwareudvikling i full-stack-løsninger. 
Vi arbejder i et fagligt stærkt miljø, hvor vi deler viden og hjælper hinanden med at udvikle os – både teknisk og personligt.
I ATP er barren sat højt, både når det gælder ambitioner og trivsel. Vi tror på et arbejdsliv i balance. Det kræver fleksibilitet med plads til den enkelte - og det har vi.
    """

In [64]:
from langchain_ollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
from typing import List
from langchain.schema import BaseMessage

class LLMClient:
    """
    Purpose:
        This client wraps Langchain's ChatOllama to invoke local LLMs via Ollama.

    Capabilities:
        - Accepts system + human messages using Langchain schema.
        - Uses Langchain's Ollama integration directly — no manual API calls.
        - Returns the final response as plain text (ready for parsing).

    Reasoning:
        - This keeps you aligned with Langchain’s native event flow.
        - You benefit from any future Langchain enhancements to `ChatOllama`.
        - You avoid any API version mismatches between you and Ollama.

    """

    def __init__(self, model: str ) -> None:
        self.llm = ChatOllama(
            model=model,
            format="json"
            )

model = ChatOllama(model="deepseek-r1:1.5b")
# class_model = LLMClient(model="deepseek-r1:1.5b")

from langchain_openai import ChatOpenAI

llm_model = ChatOpenAI(
    api_key="sk-proj-eVPJa885ae9aQ6dLTOQ7T3BlbkFJ9RElN72FpTcwNWI6r7uO",
    seed = 66,
    model="gpt-4o-2024-11-20"
)

print((llm_model))
# print(type(class_model))

client=<openai.resources.chat.completions.completions.Completions object at 0x7f38f664dba0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f38f6653a60> root_client=<openai.OpenAI object at 0x7f38f664ce20> root_async_client=<openai.AsyncOpenAI object at 0x7f38f64ba5f0> model_name='gpt-4o-2024-11-20' model_kwargs={} openai_api_key=SecretStr('**********') seed=66


In [65]:
# backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/core/data_models/analysis_result_model.py

from typing import Dict
from pydantic import BaseModel, Field

class JobAnalysisResult(BaseModel):
    company_name: str = Field(description="Identified company name")
    job_title: str = Field(description="Identified job title")
    analysis_output: str = Field(description="Analysis of the vacancy")
    employees_skills_requirement: dict = Field(description="identified skills and technical experience required for the job vacancy")
    matching_skills: dict = Field(description="matching skills in the job vacancy")


In [86]:
# backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/core/company_analysis/components/analysis_prompt_builder.py

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

class JobAnalysisResult(BaseModel):
    company_name: str = Field(description="Identified company name")
    job_title: str = Field(description="Identified job title")
    analysis_output: str = Field(description="Analysis of the vacancy")
    employees_skills_requirement: dict = Field(description="identified skills and technical experience required for the job vacancy")
    matching_skills: dict = Field(description="matching skills in the job vacancy")



def build_prompt(jd, s, parser: PydanticOutputParser) -> ChatPromptTemplate:

        # System message enforcing JSON output
        system_analysis_template_str = """
        You are an AI assistant specializing in HR job analysis.
        Your task is to analyze a given job vacancy and match it with a candidate's skills.
        - Identified relevant skills from the job description.
        - Match the required skills with the candidate’s skills.
        - Assess the candidate's suitability for the role.
        - Identify:
            - The company name.
            - The job title.
            - Required skills and technical experience (stored as a dictionary).
            - Matching skills (stored as a dictionary).
        - Write a detailed analysis of the job vacancy and the candidate's skills on a one pager
        The candidate's skills are:
        {my_skills}

        {format_instructions}
        """

        SYSTEM_PROMPT = SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                template=system_analysis_template_str,
                input_variables=["my_skills"],  # No user input needed
                partial_variables={"format_instructions": parser.get_format_instructions()}  # Enforce JSON format
            )
        )

        # Human input message
        human_analysis_template_str = """
        Here is a job description that needs analysis:
        Job Vacancy: 
        {job_position}
        
        {format_instructions}
        """

        HUMAN_PROMPT = HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                template=human_analysis_template_str,
                input_variables=["job_position"],
                partial_variables={"format_instructions": parser.get_format_instructions()}  # Enforce JSON format
            )
        )
        # Ensure that messages are formatted BEFORE returning the ChatPromptTemplate
        messages = [SYSTEM_PROMPT, HUMAN_PROMPT]
        print(messages)
        chat_prompt = ChatPromptTemplate(
            messages=messages
        )

        derp = parser.get_format_instructions()

        chat_prompt.format_messages(
            job_position=jd,
            my_skills=s,
            format_messages=derp
        )

        return chat_prompt

parser = PydanticOutputParser(pydantic_object=JobAnalysisResult)


prompt = build_prompt(job_description, skills, parser)
print(type(prompt))
print(prompt)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['my_skills'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"company_name": {"description": "Identified company name", "title": "Company Name", "type": "string"}, "job_title": {"description": "Identified job title", "title": "Job Title", "type": "string"}, "analysis_output": {"description": "Analysis of the vacancy", "title": "Analysis Output", "type": "string"}, "employees_skills_requirement": {"description": "identified skills and technica

In [ ]:

chain = prompt | llm_model | parser

structured_llm = chain.invoke(
    {
        "job_position": job_description,
        "my_skills": skills,
    }
)
print(structured_llm,"\n")


company_name='ATP' job_title='Data Scientist - Fraud Detection & AI Solutions' analysis_output="The job vacancy at ATP for a 'Data Scientist - Fraud Detection & AI Solutions' primarily focuses on leveraging data science techniques, machine learning, and NLP to detect fraud and optimize processes in the welfare system. Key responsibilities include understanding business requirements, developing machine learning models using Python, monitoring and maintaining these models, and developing and implementing NLP solutions. The job also involves a collaborative working environment in an agile (SCRUM) setup. The candidate's skills largely match the role's requirements, especially in Python, SQL, NLP, Machine Learning, and collaborative working styles. However, the role specifies a preferred background in mathematics, statistics, or related fields and familiarity with 'unsupervised methods and computer vision,' which are not explicitly mentioned among the candidate's current skills. Overall, th

In [96]:
structured_llm.analysis_output

"The job vacancy at ATP for a 'Data Scientist - Fraud Detection & AI Solutions' primarily focuses on leveraging data science techniques, machine learning, and NLP to detect fraud and optimize processes in the welfare system. Key responsibilities include understanding business requirements, developing machine learning models using Python, monitoring and maintaining these models, and developing and implementing NLP solutions. The job also involves a collaborative working environment in an agile (SCRUM) setup. The candidate's skills largely match the role's requirements, especially in Python, SQL, NLP, Machine Learning, and collaborative working styles. However, the role specifies a preferred background in mathematics, statistics, or related fields and familiarity with 'unsupervised methods and computer vision,' which are not explicitly mentioned among the candidate's current skills. Overall, the candidate is a strong fit for the role, with opportunities to align further through experienc

In [94]:
structured_llm.content
print(structured_llm.content)






AttributeError: 'JobAnalysisResult' object has no attribute 'content'